In [1]:
import pandas as pd
import numpy as np

In [3]:
seeds = [0, 25, 50, 75, 100]
models = ['A2C', 'PPO', 'DDPG', 'SAC']

## portfolio values

In [33]:
for model in models:    
    data = pd.DataFrame()
    for seed in seeds:
        data[f'portfolio_values_{seed}'] = pd.read_csv(f'tables/{model.lower()}_multi_seed/{model}_seed_{seed}_portfolio.csv', index_col='date', usecols=['date','portfolio_value'])

    data[f'{model.lower()}_mean'] = data.mean(axis=1, numeric_only=True)
    data[f'{model.lower()}_sd'] = data.std(axis=1,numeric_only=True)
    data.to_csv(f'tables/{model.lower()}_multi_seed/{model}_cumulative_period_portfolio_values.csv')

## weights

## turnovers

## transaction costs